Term Project - CS329E

1 Description

The goal of this project assignment is to develop a data analytics project using a real-world
dataset. Students should select a public dataset, think about a research project, for example,
t-test, two-sample hypothesis tests, regression models, classification, clustering, or any data
analytic task, implement the data model (using libraries or without libraries), evaluate, and
interpret the results. You can apply more than one data model.

2 Public Dataset List

Select a data set from the available public datasets.
You can find a list of such public data sets here:
http://www.teymourian.de/public-data-sets-for-data-analytic-projects/)
    
• The data set must not be very Large.

• You can reuse one of the assignment data sets.

3 Define a Project (4 Points)

Define a data science research question based on the selected dataset. You can select one of
the listed datasets or search on the web, and pick up one of the available public datasets.
You should describe the following items:

1. What is your data set about?

2. Clean up your data and reduce it to no more than 2000 observations if your data set is
very large.

3. What is exactly your research question? What do you want to learn from data? What
is your learning model ,e.g., a Classification, Clustering, etc?

4. What is your current exception about the results?

5. How do you want to evaluate your project? How to access the correctness of your
model? How well would you expect that the model will work?

4 Implementation (6 Points)

• You need to implement your project in python. You are allowed to use any Machine
Learning Library like scikit-learn1. You are also allowed to implement your project
without using any libraries.

• You can use Jupyter Notebooks to implement your project and provide your documen-
tations.

• You code should be completed and be compilable without any errors. We should be
able to read your documents, and be able to run your project.

• Run your implementation (on your Laptop or on Cluster if data is large) and generate
the results.
• Provide the interpretations of your results.
• What can you do to improve your results? Apply your ideas to improve your results.
• Provide any references that you have used.

5 Create Recorded Presentation Video of your Project (5 Points)

• Create a document or a presentation (You can create a presentation using your Jupyter
notebook, or create a powerpoint presentation, or other formats) to describe your
project and results

• Describe your code.

• Describe the results of your project in a professional way.

• You may want to visualization diagrams and describe the results based on some dia-
grams - but having diagrams is not a MUST have to get the full credit.

• Describe the model and results of your project in a way that every person in this field
can read, enjoy and understand.
Record your presentation in form of a video recording. Present your talk on your laptop
computer and record it using a desktop recording software.
You should use your webcam so that we can identify who is presenter.
You have the following 3 options to create a presentation video:

• Method 1: You can use the Panopto Deskop Recorder. You can find this software
and documentations here https://www.panopto.com/.

• Method 2: You can record your presentation using ZOOM client application, start
your zoom client, start a zoom meeting (you will be the only participant), click on
share your desktop, turn on camera and record your presentation, end the zoom session
and you will get a MP4 video file of your presentation. Upload the file to Blackboard
system.

• Method 3: Alternatively, you can use any desktop recording software, generate a MP4
file and upload MP4 to blackboard, or upload it to some other cloud storage (like google
Drive, DropBox) and share with us the URL Link of your presentation (submit the link
in Blackboard).

Note 1: Your video presentation should be between 8 minutes to maximum 15 minutes
long (Min 8 mi, Max 15 min)

Note 2: You should turn on your Web Camera so that we can see your face during the
presentation, and be able to identify the presenter.
Grading will be based on quality of your presentation, and correct describing of algorithms
or concepts.

hello
